# step1

1. Document Loader로 문서 로드
2. Text Splitter로 청킹 (CharacterTextSplitter vs RecursiveCharacterTextSplitter)
3. OpenAI/HuggingFace Embedding 생성
4. Chroma/FAISS에 벡터 저장
5. 유사도 검색 구현

In [1]:
import os
import re
import json
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.documents import Document
import numpy as np

load_dotenv()


/Users/user/Documents/study/learning_langchain/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
gpt_api_key = os.getenv("GPT_API_KEY")
llm = None

if gpt_api_key:
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        api_key=gpt_api_key
    )
    print("GPT LLM 초기화 완료")
else:
    print("GPT_API_KEY가 없어서 LLM을 사용하지 않습니다.")

llm

GPT LLM 초기화 완료


ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x15126d730>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x151712c60>, root_client=<openai.OpenAI object at 0x106011a90>, root_async_client=<openai.AsyncOpenAI object at 0x150daf9e0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

## 1. Document Loader로 문서 로드

PDF와 HTML 파일을 모두 로드하여 결합합니다. HTML에서 표를 정확하게 파싱하고, PDF에서 일반 텍스트를 추출합니다.


In [3]:
def extract_header_from_prompt(llm_prompt):
    """llm_prompt에서 헤더 정보를 추출합니다."""
    if not llm_prompt:
        return ''
    
    header_marker = "표 첫 번째 열 (맨 왼쪽):"
    summary_marker = "Summary"
    
    if header_marker in llm_prompt:
        start_idx = llm_prompt.find(header_marker) + len(header_marker)
        end_idx = llm_prompt.find(summary_marker, start_idx)
        
        if end_idx == -1:
            header = llm_prompt[start_idx:].strip()
        else:
            header = llm_prompt[start_idx:end_idx].strip()
        
        return header
    
    return ''

def load_tables_from_json(json_path):
    """table_descriptions.json 파일을 로드하여 표 Document 리스트를 생성합니다."""
    with open(json_path, 'r', encoding='utf-8') as f:
        table_data = json.load(f)
    
    table_docs = []
    
    for table_id, table_info in table_data.items():
        original_table = table_info.get('original', '')
        caption = table_info.get('caption', '')
        row_count = table_info.get('row_count', 0)
        context_before = table_info.get('context_before', [])
        llm_description = table_info.get('llm_description', '')
        llm_prompt = table_info.get('llm_prompt', '')
        
        if not original_table:
            continue
        
        context_before_str = '\n'.join(context_before) if isinstance(context_before, list) else str(context_before) if context_before else ''
        header = extract_header_from_prompt(llm_prompt)
        
        page_content_parts = []
        
        if llm_description:
            page_content_parts.append(llm_description)
        
        if context_before_str:
            page_content_parts.append(f"Context: {context_before_str}")
        
        if header:
            page_content_parts.append(f"Table Headers:\n{header}")
        
        if page_content_parts:
            page_content = '\n\n'.join(page_content_parts)
        else:
            page_content = original_table
        
        table_docs.append(Document(
            page_content=page_content,
            metadata={
                'source': json_path,
                'type': 'table',
                'table_id': table_id,
                'original_table': original_table,
                'caption': caption,
                'row_count': row_count,
                'context_before': context_before_str
            }
        ))
    
    return table_docs

pdf_path = "../data/tsla-20250930-gen.pdf"
table_json_path = "../output/table_descriptions.json"

print("="*60)
print("1. PDF 파일 로드")
print("="*60)
pdf_loader = PyPDFLoader(pdf_path)
pdf_documents = pdf_loader.load()

for doc in pdf_documents:
    doc.page_content = re.sub(r'\s+', ' ', doc.page_content)

print(f"PDF 파일 로드 완료: {pdf_path}")
print(f"PDF 문서 수: {len(pdf_documents)}")
print(f"PDF 텍스트 길이: {sum([len(doc.page_content) for doc in pdf_documents])} 문자")

print("\n" + "="*60)
print("2. JSON 파일에서 표 정보 로드")
print("="*60)

table_docs = load_tables_from_json(table_json_path)

print(f"JSON에서 로드된 표 개수: {len(table_docs)}")
if table_docs:
    print(f"표 요약 총 길이: {sum([len(doc.page_content) for doc in table_docs])} 문자")
    print(f"표 원본 총 길이: {sum([len(doc.metadata.get('original_table', '')) for doc in table_docs])} 문자")
    print(f"\n첫 번째 표 요약 (임베딩용):")
    print(table_docs[0].page_content[:500])
    print(f"\n첫 번째 표 원본 (metadata에 저장):")
    print(table_docs[0].metadata.get('original_table', '')[:500])

print("\n" + "="*60)
print("3. PDF와 표 정보 결합")
print("="*60)
documents = pdf_documents + table_docs

print(f"최종 문서 수: {len(documents)}")
print(f"  - PDF 문서: {len(pdf_documents)}")
print(f"  - 표 문서: {len(table_docs)}")
print(f"최종 텍스트 총 길이: {sum([len(doc.page_content) for doc in documents])} 문자")


1. PDF 파일 로드
PDF 파일 로드 완료: ../data/tsla-20250930-gen.pdf
PDF 문서 수: 42
PDF 텍스트 길이: 133301 문자

2. JSON 파일에서 표 정보 로드
JSON에서 로드된 표 개수: 50
표 요약 총 길이: 37222 문자
표 원본 총 길이: 31413 문자

첫 번째 표 요약 (임베딩용):
The table lists entities that are incorporated or organized in Texas, outlining their jurisdiction of incorporation. It serves to provide a clear overview of the businesses or organizations operating under Texas law, which may include various corporate details such as their names and types. This information is useful for understanding the corporate landscape in Texas and identifying the legal entities that are registered in the state.

Table Headers:
Texas
(State or other jurisdiction ofincorpor

첫 번째 표 원본 (metadata에 저장):
Texas | 91-2197729
(State or other jurisdiction of | incorporation or organization) | (I.R.S. Employer | Identification No.)

3. PDF와 표 정보 결합
최종 문서 수: 92
  - PDF 문서: 42
  - 표 문서: 50
최종 텍스트 총 길이: 170523 문자


In [4]:
print(table_docs[10].page_content)

The table presents the consolidated statements of redeemable noncontrolling interests and equity for a three-month period ending September 30, 2024, outlining changes in equity related to various financial activities. Key entries include the balance at the beginning of the period, transactions such as the settlement of warrants and stock-based compensation, distributions to noncontrolling interests, net income earned, and other comprehensive income, culminating in the total balance as of September 30, 2024. This summary highlights the company's equity dynamics and how specific activities influence overall equity and noncontrolling interests within that timeframe.

Context: Consolidated Statements of Redeemable Noncontrolling Interests and Equity (in millions) (unaudited)

Table Headers:
Three Months Ended September 30, 2024
Balance as of June 30, 2024
Settlement of warrants
Issuance of common stock for equity incentive awards
Stock-based compensation
Distributions to noncontrolling int

In [5]:
len(table_docs[10].page_content)

1077

In [6]:
if len(table_docs) > 10:
    print(table_docs[10].page_content)
else:
    print(f"표 문서가 {len(table_docs)}개만 있습니다.")

The table presents the consolidated statements of redeemable noncontrolling interests and equity for a three-month period ending September 30, 2024, outlining changes in equity related to various financial activities. Key entries include the balance at the beginning of the period, transactions such as the settlement of warrants and stock-based compensation, distributions to noncontrolling interests, net income earned, and other comprehensive income, culminating in the total balance as of September 30, 2024. This summary highlights the company's equity dynamics and how specific activities influence overall equity and noncontrolling interests within that timeframe.

Context: Consolidated Statements of Redeemable Noncontrolling Interests and Equity (in millions) (unaudited)

Table Headers:
Three Months Ended September 30, 2024
Balance as of June 30, 2024
Settlement of warrants
Issuance of common stock for equity incentive awards
Stock-based compensation
Distributions to noncontrolling int

## 2. Text Splitter로 청킹 비교

CharacterTextSplitter와 RecursiveCharacterTextSplitter를 비교합니다.

`CharacterTextSplitter`와 `RecursiveCharacterTextSplitter`의 차이:



### 주요 차이점 비교

| 특징 | CharacterTextSplitter | RecursiveCharacterTextSplitter |
|------|----------------------|-------------------------------|
| **분할 방식** | 단일 구분자만 사용 | 여러 구분자 우선순위 적용 |
| **청크 크기 보장** | 보장 안 됨 (구분자 없으면 초과 가능) | 최대한 보장 (재귀적 분할) |
| **문맥 보존** | 낮음 | 높음 |
| **속도** | 빠름 | 상대적으로 느림 |
| **사용 사례** | 구조화된 텍스트 | 일반 텍스트, 문서 |
| **권장 사용** | 특정 구분자가 확실할 때 | 일반적인 경우 |

## 실제 동작 예시

**CharacterTextSplitter:**
```text
텍스트: "문장1\n문장2\n문장3" (총 200자)
구분자: "\n"
→ ["문장1", "문장2", "문장3"] (각각 200자일 수 있음)
```

**RecursiveCharacterTextSplitter:**
```text
텍스트: "문단1\n\n문단2\n\n문단3" (총 200자)
→ 먼저 "\n\n"로 시도
→ 실패하면 "\n"로 시도
→ 실패하면 " "로 시도
→ 최종적으로 chunk_size에 맞게 분할
```


In [7]:
chunk_size = 500
chunk_overlap = 50

char_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator="\n"
)

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

test_doc = documents[0].page_content

char_chunks = char_splitter.split_text(test_doc)
recursive_chunks = recursive_splitter.split_text(test_doc)

print(f"원본 문서 길이: {len(test_doc)} 문자")
print(f"\nCharacterTextSplitter:")
print(f"  청크 개수: {len(char_chunks)}")
print(f"  평균 청크 길이: {np.mean([len(chunk) for chunk in char_chunks]):.0f} 문자")
print(f"  첫 번째 청크 길이: {len(char_chunks[0])} 문자")
print(f"\nRecursiveCharacterTextSplitter:")
print(f"  청크 개수: {len(recursive_chunks)}")
print(f"  평균 청크 길이: {np.mean([len(chunk) for chunk in recursive_chunks]):.0f} 문자")
print(f"  첫 번째 청크 길이: {len(recursive_chunks[0])} 문자")

print(f"\n첫 번째 청크 샘플 (CharacterTextSplitter):")
print(char_chunks[0][:200] + "...")
print(f"\n첫 번째 청크 샘플 (RecursiveCharacterTextSplitter):")
print(recursive_chunks[0][:200] + "...")


원본 문서 길이: 2587 문자

CharacterTextSplitter:
  청크 개수: 1
  평균 청크 길이: 2587 문자
  첫 번째 청크 길이: 2587 문자

RecursiveCharacterTextSplitter:
  청크 개수: 6
  평균 청크 길이: 469 문자
  첫 번째 청크 길이: 498 문자

첫 번째 청크 샘플 (CharacterTextSplitter):
UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549 FORM 10-Q (Mark One) x QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the quarterly...

첫 번째 청크 샘플 (RecursiveCharacterTextSplitter):
UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549 FORM 10-Q (Mark One) x QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the quarterly...


In [8]:
# 표 제목과 표 내용이 분리되지 않도록 처리
def is_table_header(line):
    table_patterns = [
        r'the following table',
        r'table \d+',
        r'^\s*table\s+\d+',
        r'표\s*\d+',
        r'다음 표',
        r'consolidated.*balance.*sheet',
        r'consolidated.*statement.*of.*operations',
        r'consolidated.*statement.*of.*cash',
        r'consolidated.*statement.*of.*income',
        r'balance sheet',
        r'statement of operations',
        r'income statement',
        r'cash flow',
        r'financial.*statement',
        r'\(unaudited\)',
        r'\(audited\)',
    ]
    
    for pattern in table_patterns:
        if re.search(pattern, line, re.IGNORECASE):
            return True
    return False

def is_table_row(line):
    if not line or len(line.strip()) < 3:
        return False
    
    line_stripped = line.strip()
    
    if '\t' in line or '|' in line:
        return True
    
    parts = re.split(r'\s{2,}|\t', line_stripped)
    if len(parts) >= 3:
        has_numbers = sum(1 for p in parts if re.search(r'[\d,\.\$\(\)]', p)) >= 2
        if has_numbers:
            return True
    
    if re.search(r'^\s*[A-Z][a-z]+.*\$\s*[\d,]+', line_stripped):
        return True
    
    if re.search(r'^\s*[A-Z][a-z]+.*\(.*\)\s*\$', line_stripped):
        return True
    
    return False

def is_table_end(line, next_lines):
    if not line:
        return False
    
    if re.search(r'^\s*note\s+\d+|^\s*see\s+note', line, re.IGNORECASE):
        return True
    
    if len(line) > 200 and re.search(r'[.!?]$', line):
        return True
    
    if next_lines and len(next_lines) > 0:
        next_line = next_lines[0].strip() if next_lines[0] else ''
        if next_line and not is_table_row(next_line) and len(next_line) > 50:
            if not is_table_header(next_line):
                return True
    
    return False

def preserve_tables_in_chunks(docs, splitter, chunk_size=500):
    all_chunks = []
    
    for doc in docs:
        text = doc.page_content
        lines = text.split('\n')
        
        processed_blocks = []
        i = 0
        
        while i < len(lines):
            line = lines[i].strip()
            
            if is_table_header(line):
                table_block = [line]
                i += 1
                consecutive_table_rows = 0
                
                while i < len(lines):
                    next_line = lines[i] if i < len(lines) else ''
                    next_line_stripped = next_line.strip()
                    
                    if not next_line_stripped:
                        table_block.append('')
                        consecutive_table_rows = 0
                        i += 1
                        continue
                    
                    if is_table_row(next_line):
                        table_block.append(next_line)
                        consecutive_table_rows += 1
                        i += 1
                    elif is_table_end(next_line_stripped, lines[i+1:i+3] if i+1 < len(lines) else []):
                        break
                    elif consecutive_table_rows > 0 and len(next_line_stripped) < 150:
                        table_block.append(next_line)
                        i += 1
                    else:
                        break
                
                processed_blocks.append('\n'.join(table_block))
            else:
                processed_blocks.append(line)
                i += 1
        
        merged_text = '\n'.join(processed_blocks)
        merged_doc = Document(page_content=merged_text, metadata=doc.metadata)
        
        chunks = splitter.split_documents([merged_doc])
        all_chunks.extend(chunks)
    
    return all_chunks

pdf_docs = [doc for doc in documents if doc.metadata.get('type') != 'table']
table_docs = [doc for doc in documents if doc.metadata.get('type') == 'table']

print(f"PDF 문서 수: {len(pdf_docs)}")
print(f"표 문서 수: {len(table_docs)}")

pdf_chunks = recursive_splitter.split_documents(pdf_docs)

all_chunks = pdf_chunks + table_docs

print(f"\n총 청크 개수: {len(all_chunks)}")
print(f"  - PDF에서 생성된 청크: {len(pdf_chunks)}")
print(f"  - JSON에서 로드된 표: {len(table_docs)}")
print(f"평균 청크 길이: {np.mean([len(chunk.page_content) for chunk in all_chunks]):.0f} 문자")

if table_docs:
    print(f"\n표 문서 샘플 (처음 500자):")
    print(table_docs[0].page_content[:500])


PDF 문서 수: 42
표 문서 수: 50

총 청크 개수: 360
  - PDF에서 생성된 청크: 310
  - JSON에서 로드된 표: 50
평균 청크 길이: 508 문자

표 문서 샘플 (처음 500자):
The table lists entities that are incorporated or organized in Texas, outlining their jurisdiction of incorporation. It serves to provide a clear overview of the businesses or organizations operating under Texas law, which may include various corporate details such as their names and types. This information is useful for understanding the corporate landscape in Texas and identifying the legal entities that are registered in the state.

Table Headers:
Texas
(State or other jurisdiction ofincorpor


## 3. Embedding 생성

OpenAI와 HuggingFace Embedding을 생성합니다.


In [ ]:
import shutil

if os.path.exists("./chroma_db"):
    shutil.rmtree("./chroma_db")
    print("기존 Chroma 저장소 삭제 완료")

if os.path.exists("./faiss_db"):
    shutil.rmtree("./faiss_db")
    print("기존 FAISS 저장소 삭제 완료")

print(f"저장할 문서 수: {len(all_chunks)}개")


In [9]:
# 모델 최대 입력길이: 512 토큰
hf_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
print("HuggingFace Embeddings 초기화 완료")

test_text = "This is a test sentence for embedding."
hf_embedding = hf_embeddings.embed_query(test_text)
print(f"HuggingFace Embedding 차원: {len(hf_embedding)}")


HuggingFace Embeddings 초기화 완료
HuggingFace Embedding 차원: 384


## 4. 벡터 저장소에 저장

Chroma와 FAISS에 벡터를 저장합니다.


In [10]:
chroma_db = Chroma.from_documents(
    documents=all_chunks,
    embedding=hf_embeddings,
    persist_directory="./chroma_db"
)
print("Chroma 벡터 저장소 생성 완료")
try:
    doc_count = chroma_db._collection.count()
    print(f"저장된 문서 수: {doc_count}")
    
    all_docs = chroma_db._collection.get()
    table_count = sum(1 for meta in all_docs.get('metadatas', []) if meta.get('type') == 'table')
    text_count = sum(1 for meta in all_docs.get('metadatas', []) if meta.get('type') != 'table')
    print(f"  - 표 문서: {table_count}개")
    print(f"  - 텍스트 문서: {text_count}개")
except:
    print(f"저장된 문서 수: {len(all_chunks)} (추정)")
    table_count_in_chunks = sum(1 for chunk in all_chunks if chunk.metadata.get('type') == 'table')
    print(f"  - 표 문서: {table_count_in_chunks}개")


Chroma 벡터 저장소 생성 완료
저장된 문서 수: 2520
  - 표 문서: 350개
  - 텍스트 문서: 2170개


In [11]:
faiss_db = FAISS.from_documents(
    documents=all_chunks,
    embedding=hf_embeddings
)
faiss_db.save_local("./faiss_db")
print("FAISS 벡터 저장소 생성 완료")
print(f"저장된 문서 수: {len(faiss_db.docstore._dict)}")

table_count_in_faiss = sum(1 for doc_id, doc in faiss_db.docstore._dict.items() if doc.metadata.get('type') == 'table')
text_count_in_faiss = len(faiss_db.docstore._dict) - table_count_in_faiss
print(f"  - 표 문서: {table_count_in_faiss}개")
print(f"  - 텍스트 문서: {text_count_in_faiss}개")


FAISS 벡터 저장소 생성 완료
저장된 문서 수: 360
  - 표 문서: 50개
  - 텍스트 문서: 310개


## 5. 유사도 검색 구현 및 테스트


In [12]:
def search_tables_only_improved(query, vectorstore, store_name, top_k=10):
    """표만 직접 검색해서 유사도가 높은 순으로 보여줍니다."""
    table_results = []
    
    try:
        if isinstance(vectorstore, Chroma):
            all_docs = vectorstore._collection.get()
            table_docs = []
            table_embeddings = []
            
            for i, metadata in enumerate(all_docs.get('metadatas', [])):
                if metadata.get('type') == 'table':
                    doc = Document(
                        page_content=all_docs['documents'][i],
                        metadata=metadata
                    )
                    table_docs.append(doc)
                    table_embeddings.append(all_docs['embeddings'][i])
            
            if table_docs:
                query_embedding = vectorstore._embedding_function.embed_query(query)
                
                for doc, doc_embedding in zip(table_docs, table_embeddings):
                    score = np.dot(query_embedding, doc_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_embedding))
                    distance = 1 - score
                    table_results.append((doc, distance))
        
        elif isinstance(vectorstore, FAISS):
            all_table_docs = []
            for doc_id, doc in vectorstore.docstore._dict.items():
                if doc.metadata.get('type') == 'table':
                    all_table_docs.append(doc)
            
            if all_table_docs:
                query_embedding = vectorstore.embeddings.embed_query(query)
                
                for doc in all_table_docs:
                    doc_embedding = vectorstore.embeddings.embed_documents([doc.page_content])[0]
                    score = np.dot(query_embedding, doc_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_embedding))
                    distance = 1 - score
                    table_results.append((doc, distance))
        
    except Exception as e:
        print(f"표 직접 검색 중 오류: {e}, 전체 검색으로 대체합니다.")
        search_k = 200
        results = vectorstore.similarity_search_with_score(query, k=search_k)
        for doc, score in results:
            if doc.metadata.get('type') == 'table':
                table_results.append((doc, score))
    
    if not table_results:
        print(f"\n{'='*60}")
        print(f"{store_name} 표 검색 결과: '{query}'")
        print(f"{'='*60}")
        print("표 검색 결과가 없습니다.")
        return []
    
    table_results.sort(key=lambda x: x[1])
    
    print(f"\n{'='*60}")
    print(f"{store_name} 표 검색 결과: '{query}'")
    print(f"{'='*60}")
    print(f"총 {len(table_results)}개의 표가 검색되었습니다.\n")
    
    for i, (doc, score) in enumerate(table_results[:top_k], 1):
        print(f"[표 결과 {i}] (유사도 점수: {score:.4f})")
        
        caption = doc.metadata.get('caption', 'N/A')
        row_count = doc.metadata.get('row_count', 'N/A')
        table_id = doc.metadata.get('table_id', 'N/A')
        original_table = doc.metadata.get('original_table', '')
        
        print(f"표 ID: {table_id}")
        print(f"표 제목: {caption}")
        print(f"표 행 수: {row_count}")
        print(f"표 내용 (임베딩용): {doc.page_content[:300]}...")
        print(f"\n표 원본:")
        print(original_table[:800] + ("..." if len(original_table) > 800 else ""))
        print("\n" + "-"*60)
    
    return table_results


In [13]:
def search_with_table_info(query, vectorstore, store_name, top_k=3):
    print(f"\n{'='*60}")
    print(f"{store_name} 검색 결과: '{query}'")
    print(f"{'='*60}")
    
    results = vectorstore.similarity_search_with_score(query, k=top_k)
    
    enriched_results = []
    
    for i, (doc, score) in enumerate(results, 1):
        print(f"\n[결과 {i}] (유사도 점수: {score:.4f})")
        
        doc_type = doc.metadata.get('type', 'text')
        page_num = doc.metadata.get('page', 'N/A')
        source = doc.metadata.get('source', 'N/A')
        
        print(f"타입: {doc_type}")
        print(f"출처: {source} (페이지: {page_num})")
        
        if doc_type == 'table':
            caption = doc.metadata.get('caption', 'N/A')
            row_count = doc.metadata.get('row_count', 'N/A')
            original_table = doc.metadata.get('original_table', '')
            
            print(f"표 제목: {caption}")
            print(f"표 행 수: {row_count}")
            print(f"표 요약 (임베딩용): {doc.page_content[:200]}...")
            print(f"\n표 원본:")
            print(original_table[:500] + ("..." if len(original_table) > 500 else ""))
            
            enriched_results.append({
                'doc': doc,
                'score': score,
                'summary': doc.page_content,
                'original_table': original_table,
                'caption': caption,
                'type': 'table'
            })
        else:
            print(f"내용: {doc.page_content[:200]}...")
            enriched_results.append({
                'doc': doc,
                'score': score,
                'content': doc.page_content,
                'type': 'text'
            })
    
    return enriched_results

def search_tables_only(query, vectorstore, store_name, top_k=10):
    """표만 필터링해서 유사도가 높은 순으로 보여줍니다."""
    search_k = max(top_k * 5, 50)
    results = vectorstore.similarity_search_with_score(query, k=search_k)
    
    table_results = []
    doc_types = {}
    for doc, score in results:
        doc_type = doc.metadata.get('type', 'text')
        doc_types[doc_type] = doc_types.get(doc_type, 0) + 1
        if doc_type == 'table':
            table_results.append((doc, score))
    
    if not table_results:
        print(f"\n{'='*60}")
        print(f"{store_name} 표 검색 결과: '{query}'")
        print(f"{'='*60}")
        print(f"표 검색 결과가 없습니다.")
        print(f"검색된 문서 타입 분포: {doc_types}")
        print(f"총 {len(results)}개 결과 중 표가 없습니다.")
        return []
    
    table_results.sort(key=lambda x: x[1])
    
    print(f"\n{'='*60}")
    print(f"{store_name} 표 검색 결과: '{query}'")
    print(f"{'='*60}")
    print(f"총 {len(table_results)}개의 표가 검색되었습니다.\n")
    
    for i, (doc, score) in enumerate(table_results[:top_k], 1):
        print(f"[표 결과 {i}] (유사도 점수: {score:.4f})")
        
        caption = doc.metadata.get('caption', 'N/A')
        row_count = doc.metadata.get('row_count', 'N/A')
        table_id = doc.metadata.get('table_id', 'N/A')
        original_table = doc.metadata.get('original_table', '')
        
        print(f"표 ID: {table_id}")
        print(f"표 제목: {caption}")
        print(f"표 행 수: {row_count}")
        print(f"표 내용 (임베딩용): {doc.page_content[:300]}...")
        print(f"\n표 원본:")
        print(original_table[:800] + ("..." if len(original_table) > 800 else ""))
        print("\n" + "-"*60)
    
    return table_results


In [14]:
test_queries = [
    "What is Tesla's revenue in Q3 2025?",
    "What are Tesla's production numbers?",
    "What is Tesla's energy business performance?",
    "What are Tesla's future plans?",
    "What is Tesla's financial outlook?"
]

print("="*60)
print("Chroma 벡터 저장소 검색 테스트")
print("="*60)

for query in test_queries:
    search_with_table_info(query, chroma_db, "Chroma", top_k=10)
    search_tables_only(query, chroma_db, "Chroma", top_k=5)


Chroma 벡터 저장소 검색 테스트

Chroma 검색 결과: 'What is Tesla's revenue in Q3 2025?'

[결과 1] (유사도 점수: 0.7442)
타입: text
출처: ../data/tsla-20250930-gen.pdf (페이지: 4)
내용: Table of Contents Tesla, Inc. Consolidated Statements of Operations (in millions, except per share data) (unaudited) Three Months Ended September 30, Nine Months Ended September 30, 2025 2024 2025 202...

[결과 2] (유사도 점수: 0.7442)
타입: text
출처: ../data/tsla-20250930-gen.pdf (페이지: 4)
내용: Table of Contents Tesla, Inc. Consolidated Statements of Operations (in millions, except per share data) (unaudited) Three Months Ended September 30, Nine Months Ended September 30, 2025 2024 2025 202...

[결과 3] (유사도 점수: 0.7442)
타입: text
출처: ../data/tsla-20250930-gen.pdf (페이지: 4)
내용: Table of Contents Tesla, Inc. Consolidated Statements of Operations (in millions, except per share data) (unaudited) Three Months Ended September 30, Nine Months Ended September 30, 2025 2024 2025 202...

[결과 4] (유사도 점수: 0.7442)
타입: text
출처: ../data/tsla-20250930-gen.pdf (페이

In [15]:
print("="*60)
print("FAISS 벡터 저장소 검색 테스트")
print("="*60)

for query in test_queries:
    search_with_table_info(query, faiss_db, "FAISS", top_k=3)


FAISS 벡터 저장소 검색 테스트

FAISS 검색 결과: 'What is Tesla's revenue in Q3 2025?'

[결과 1] (유사도 점수: 0.7442)
타입: text
출처: ../data/tsla-20250930-gen.pdf (페이지: 4)
내용: Table of Contents Tesla, Inc. Consolidated Statements of Operations (in millions, except per share data) (unaudited) Three Months Ended September 30, Nine Months Ended September 30, 2025 2024 2025 202...

[결과 2] (유사도 점수: 0.7699)
타입: text
출처: ../data/tsla-20250930-gen.pdf (페이지: 1)
내용: TESLA, INC. FORM 10-Q FOR THE QUARTER ENDED SEPTEMBER 30, 2025 INDEX Page PART I. FINANCIAL INFORMATION Item 1. Financial Statements 4 Consolidated Balance Sheets 4 Consolidated Statements of Operatio...

[결과 3] (유사도 점수: 0.7996)
타입: text
출처: ../data/tsla-20250930-gen.pdf (페이지: 41)
내용: of Section 13(a) or 15(d) of the Securities Exchange Act of 1934 and (ii) that the information contained in such Form 10-Q fairly presents, in all material respects, the financial condition and result...

FAISS 검색 결과: 'What are Tesla's production numbers?'

[결과 1] (유사도 점수: 0.

## 6. GPT LLM을 사용한 RAG 테스트

검색된 문서를 컨텍스트로 사용하여 GPT LLM으로 답변 생성합니다.


In [16]:
if llm:
    def rag_query(query, vectorstore, top_k=3):
        docs = vectorstore.similarity_search(query, k=top_k)
        
        context_parts = []
        table_info = []
        
        for doc in docs:
            if doc.metadata.get('type') == 'table':
                original_table = doc.metadata.get('original_table', doc.page_content)
                caption = doc.metadata.get('caption', '')
                context_parts.append(f"[표: {caption}]\n{original_table}")
                table_info.append({
                    'caption': caption,
                    'original_table': original_table,
                    'summary': doc.page_content
                })
            else:
                context_parts.append(doc.page_content)
        
        context = "\n\n".join(context_parts)
        
        prompt = f"""다음 컨텍스트를 바탕으로 질문에 답변해주세요.

컨텍스트:
{context}

질문: {query}

답변:"""
        
        response = llm.invoke(prompt)
        return response.content, docs, table_info
    
    test_query = "What is Tesla's Q3 2025 financial performance and key highlights?"
    print("="*60)
    print("GPT RAG 테스트")
    print("="*60)
    print(f"질문: {test_query}\n")
    
    answer, retrieved_docs, table_info = rag_query(test_query, chroma_db)
    print("답변:")
    print(answer)
    print(f"\n참고한 문서 수: {len(retrieved_docs)}")
    
    if table_info:
        print(f"\n참고한 표 수: {len(table_info)}")
        for i, table in enumerate(table_info, 1):
            print(f"\n[표 {i}] {table['caption']}")
            print(f"원본 표 (처음 300자):")
            print(table['original_table'][:300] + "...")
else:
    print("GPT LLM이 초기화되지 않았습니다. .env 파일에 GPT_API_KEY를 설정해주세요.")


GPT RAG 테스트
질문: What is Tesla's Q3 2025 financial performance and key highlights?

답변:
제가 가진 정보는 2023년 10월까지의 데이터에 기반하고 있으며, Tesla의 2025년 3분기(Q3) 재무 성과 및 주요 하이라이트에 대한 구체적인 정보는 제공할 수 없습니다. 해당 정보는 공식적인 재무 보고서나 뉴스 자료를 통해 확인하는 것이 가장 정확합니다. Tesla의 Q3 2025 성과에 대한 질문은 해당 시점의 공식 보고서를 참고하시기 바랍니다.

참고한 문서 수: 3
